In [1]:
# nn
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
%cd ..

/Users/cheriehe/OneDrive - Business/kaggle202/earthquake prediction


In [3]:
%cd ..

/Users/cheriehe/OneDrive - Business/kaggle202


In [4]:
scaled_test_X=pd.read_csv('scale_test_features_new.csv')

In [5]:
scaled_test_X.head()

,Unnamed: 0,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,...,std_roll_std_10000,std_roll_std_50,std_roll_std_500,sum,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_5,time_rev_asym_stat_50,trend
0,0,0.706720,0.706511,0.707966,0.706740,0.042268,0.012960,0.013842,0.006158,0.013459,...,0.008027,0.013292,0.012711,0.706678,0.554595,0.649700,0.618910,0.548292,0.256051,0.500608
1,1,0.497303,0.497618,0.498812,0.497252,0.045164,0.018472,0.015254,0.007911,0.008635,...,0.013158,0.021184,0.020651,0.497219,0.554611,0.649709,0.618912,0.548306,0.256061,0.679936
2,2,0.784127,0.784395,0.786238,0.784101,0.052868,0.032772,0.035384,0.016449,0.020794,...,0.023339,0.029346,0.028924,0.784078,0.554633,0.649722,0.618909,0.548330,0.256051,0.548720
3,3,0.732647,0.732740,0.734171,0.732616,0.028412,0.009683,0.008434,0.004689,0.012972,...,0.006785,0.010980,0.010774,0.732609,0.554597,0.649699,0.618917,0.548292,0.256050,0.505784
4,4,0.469226,0.468777,0.465227,0.469259,0.040391,0.022196,0.016794,0.010551,0.008814,...,0.015242,0.021200,0.020740,0.469250,0.554580,0.649706,0.618924,0.548285,0.256051,0.396607


In [6]:
scaled_test_X.columns

Index(['Unnamed: 0', 'Hann_window_mean_150', 'Hann_window_mean_1500',
       'Hann_window_mean_15000', 'Hann_window_mean_50', 'Hilbert_mean',
       'abs_max', 'abs_max_roll_mean_10', 'abs_max_roll_mean_100',
       'abs_max_roll_mean_1000',
       ...
       'std_roll_std_10000', 'std_roll_std_50', 'std_roll_std_500', 'sum',
       'time_rev_asym_stat_1', 'time_rev_asym_stat_10',
       'time_rev_asym_stat_100', 'time_rev_asym_stat_5',
       'time_rev_asym_stat_50', 'trend'],
      dtype='object', length=828)

In [7]:
scaled_test_X.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
scaled_test_X.shape

(2624, 827)

In [9]:
scaled_train_X=pd.read_csv('scale_train_features_new.csv')
train_y=pd.read_csv('y_features.csv')

In [10]:
scaled_train_X.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
scaled_train_X.shape

(4195, 827)

In [12]:
train_y.shape

(4195, 1)

In [13]:
y_train_flatten=train_y.values.ravel()

In [14]:
x_train_scaled1, x_test_scaled1, y_train1, y_test1 = train_test_split(scaled_train_X, train_y, test_size = 0.2, random_state = 101)

In [15]:
y_train_flatten1 = y_train1.values.ravel() #flattening the y_train
y_test_flatten1 = y_test1.values.ravel() #flattening the y_test

In [16]:
print(x_train_scaled1.shape)
print(x_test_scaled1.shape)
print(y_train_flatten1.shape)
print(y_test_flatten1.shape)

(3356, 827)
(839, 827)
(3356,)
(839,)


In [17]:
#1. BLOCK1: ((DENSE(512 filters) ---> ACTIVATION ---> BN) x 2) -----> DROPOUT

#2. BLOCK1: ((DENSE(32 filters) ---> ACTIVATION ---> BN) x 2) -----> DROPOUT

#3. BLOCK2 : ((DENSE(32 filters) ---> ACTIVATION ---> BN) x 2) -----> DROPOUT

#4. BLOCK3: ((DENSE(32 filters) ---> ACTIVATION ---> BN) x 2) -----> DROPOUT

#5. BLOCK4(output layer): (DENSE(classes) ---> ACTIVATION 2)

In [18]:
import keras
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import Sequential
from keras import optimizers

Using TensorFlow backend.


In [19]:
kernel_init = 'he_normal'
def Model(input_dim, activation, classes):
    model = Sequential()

    model.add(Dense(512, kernel_initializer = kernel_init, input_dim = input_dim))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dense(32, kernel_initializer = kernel_init, input_dim = input_dim))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(32, kernel_initializer = kernel_init)) 
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dense(32, kernel_initializer = kernel_init)) 
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.01))

    model.add(Dense(32, kernel_initializer = kernel_init))    
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dense(32, kernel_initializer = kernel_init))    
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.001))
    
    model.add(Dense(32, kernel_initializer = kernel_init))    
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dense(32, kernel_initializer = kernel_init))    
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.0001))

    model.add(Dense(classes, kernel_initializer = kernel_init))    
    model.add(Activation('linear'))
    
    return model

In [20]:
input_dim = scaled_train_X.shape[1]
activation = 'tanh'
classes = 1 #the output labels
model = Model(input_dim = input_dim, activation = activation, classes = classes)
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
optim = optimizers.Adam(lr = 0.001, decay = 0.001 / 32)
model.compile(loss = 'mean_absolute_error', optimizer = optim, metrics = ['mean_absolute_error'])

In [22]:
mae_list=[]
import sys
for i in range(1,12):
    sys.stdout = open('keras_output6.txt', 'a')
    h= model.fit(x_train_scaled1, y_train_flatten1, epochs = i, batch_size = 32, verbose = 1)
    sys.stdout = sys.__stdout__
    predictions = model.predict(x_test_scaled1, verbose = 1, batch_size = 32) #make predictons on the test data 
    mae = mean_absolute_error(predictions, y_test_flatten1) 
    mae_list.append(mae)

Instructions for updating:
Use tf.cast instead.


In [23]:
mae_list

[4.544098884881387,
 2.874447800960509,
 2.718986084662533,
 2.8541202856867876,
 3.120365549479137,
 2.7601288877080226,
 2.8476649035439,
 2.4211986714801785,
 3.107544913320667,
 2.42965324644907,
 2.955813910506325]

# epochs, batch_size

In [24]:
input_dim = scaled_test_X.shape[1]
activation = 'tanh'
classes = 1

history = dict() #dictionery to store the history of individual models for later visualization
prediction_scores = dict() #dictionery to store the predicted scores of individual models on the test dataset

#here we will be training the same model for a total of 10 times and will be considering the mean of the output values for predictions
for i in np.arange(0, 15):
    optim = optimizers.Adam(lr = 0.001, decay = 0.001 / 32)
    ensemble_model = Model(input_dim = input_dim, activation = activation, classes = classes)
    ensemble_model.compile(loss = 'mean_absolute_error', optimizer = optim, metrics = ['mean_absolute_error'])
    print('TRAINING MODEL NO : {}'.format(i))
    sys.stdout = open('keras_output7.txt', 'w')
    H = ensemble_model.fit(scaled_train_X, y_train_flatten,
                  batch_size = 32,
                  epochs = 11,
                  verbose = 1)
    sys.stdout = sys.__stdout__
    history[i] = H
    
#     ensemble_model.save('MODEL_{}.model'.format(i))
    
    predictions = ensemble_model.predict(scaled_test_X, verbose = 1, batch_size = 32)
    prediction_scores[i] = predictions

In [25]:
#making predictions
prediction1 = np.hstack([p.reshape(-1,1) for p in prediction_scores.values()]) #taking the scores of all the trained models
prediction1 = np.mean(prediction1, axis = 1)

print(prediction1.shape)

In [28]:
prediction1

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [26]:
#submitting the file
submission = pd.read_csv('sample_submission.csv', index_col = 'seg_id')
submission['time_to_failure'] = prediction1
submission.to_csv('submission_nn3.csv')